# 6. Klassifikation - Modeling
#### Überwachtes Klassifikationsverfahren
## Anforderungen an Projektumsetzung: Klassifikation

---
**AUFGABE:**

- Führen Sie mit dem Algorithmus Ihrer Wahl eine Klassifikationsaufgabe auf Ihren Daten durch.
- Teilen Sie dazu zunächst die Daten auf, um Overfitting beim Trainieren des Algorithmus und bei der Parameterauswahl zu vermeiden. Erklären Sie die gewählte Strategie und die Größenverhältnisse.
- Wählen Sie geeignete Features aus und setzen Sie die Parameter des Algorithmus. Beschreiben Sie das gewälhte Vorgehen für die Auswahl der Features und Parameter. Berichten Sie den Parameterraum und die final gewählten Parameter. Geben Sie die Performanz auf den Trainingsdaten (bzw. Entwicklungsdaten, falls verwendet) an.
- Evaluieren Sie die Klassifikation auf den ungesehenen Testdaten. Betrachten Sie Precision und Recall sowie den F-Wert. Welches Maß ist für Ihre Anwendung wichtiger? Bewerten Sie Ihr Ergebnis. Ist es in der Praxis voraussichtlich zufriedenstellend?

In [1]:
# Imports für unten

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd

In [2]:
apps = pd.read_csv("Daten/Google-Playstore_Edit2.csv")

In [3]:
# alle Kategorien löschen, die für uns als Unternehmen irrelevant und für eine Entwicklung ausgeschlossen sind
less_apps = apps[apps['Category'] == 'Action'] + apps[apps['Category'] == 'Arcade'] + apps[apps['Category'] == 'Beauty'] + apps[apps['Category'] == 'Casino'] + apps[apps['Category'] == 'Comics'] + apps[apps['Category'] == 'Dating'] + apps[apps['Category'] == 'Educational'] + apps[apps['Category'] == 'Puzzle'] + apps[apps['Category'] == 'Racing'] + apps[apps['Category'] == 'Role Playing'] + apps[apps['Category'] == 'Shopping'] + apps[apps['Category'] == 'Trivia'] + apps[apps['Category'] == 'Video Players & Editors'] 
            
apps.drop(less_apps.index, axis=0, inplace=True)            

In [4]:
cat = apps.groupby('Category')
cat['Category'].count()

Category
Adventure             23196
Art & Design          18538
Auto & Vehicles       18278
Board                 10588
Books & Reference    116726
Business             143761
Card                   8179
Casual                50797
Communication         48159
Education            241075
Entertainment        138268
Events                12839
Finance               65456
Food & Drink          73920
Health & Fitness      83501
House & Home          14369
Libraries & Demo       5196
Lifestyle            118324
Maps & Navigation     26722
Medical               32063
Music                  4202
Music & Audio        154898
News & Magazines      42804
Parenting              3810
Personalization       89210
Photography           35552
Productivity          79686
Simulation            23276
Social                44729
Sports                47478
Strategy               8525
Tools                143976
Travel & Local        67282
Weather                7245
Word                   8630
Name: Categ

In [5]:
# Datensatz random auf die Hälfte reduzieren und neues DataFrame erstellen
half_apps = apps.sample(frac = 0.5)

In [6]:
# Alle Spalten mit Unique-Werten werden gedropped - zu viel Rechenkapa notwendig
# apps_v2 = apps.copy()
half_apps.drop(columns=['App Name', 'App Id', 'Developer Id', 'Developer Website','Minimum Android', 'Developer Email', 'Privacy Policy', 'Released', 'Scraped Time', 'Last Updated'], inplace=True)

In [7]:
# Umwandlung in Float-Werte
half_apps['Free']             = half_apps['Free'].astype(float)
half_apps['Ad Supported']     = half_apps['Ad Supported'].astype(float)
half_apps['Editors Choice']   = half_apps['Editors Choice'].astype(float)
half_apps['In App Purchases'] = half_apps['In App Purchases'].astype(float)
half_apps['Maximum Installs'] = half_apps['Maximum Installs'].astype(float)

In [8]:
half_apps.dtypes

Category             object
Rating              float64
Rating Count        float64
Installs             object
Minimum Installs    float64
Maximum Installs    float64
Free                float64
Price               float64
Currency             object
Size                float64
Content Rating       object
Ad Supported        float64
In App Purchases    float64
Editors Choice      float64
Released Year       float64
dtype: object

In [9]:
# Selektion von den Spalten vom Typ object
half_apps = half_apps.dropna()
half_apps.select_dtypes(include=['object'])

,Category,Installs,Currency,Content Rating
1079365,Sports,"5,000+",USD,Everyone
788722,Health & Fitness,"1,000+",USD,Everyone
2109951,Lifestyle,1+,USD,Everyone
807712,Sports,"1,000+",USD,Everyone
1895366,Personalization,"1,000+",USD,Everyone
...,...,...,...,...
1323376,Books & Reference,10+,USD,Everyone
1850523,Communication,100+,USD,Everyone
90599,Music & Audio,100+,USD,Everyone
638084,Adventure,"1,000+",USD,Everyone


In [10]:
# Aufteilung in Listen mit numerischen und mit noch kategorischen Werten
numerical_cols = list(half_apps.select_dtypes(include="float").columns)
categorical_cols = list(half_apps.select_dtypes(include="object").columns)

In [11]:
# Löschen von Category, da dies dann als Zielklasse verwendet werden soll
categorical_cols.remove("Category")
categorical_cols

['Installs', 'Currency', 'Content Rating']

In [12]:
# Da Klassifikation nur mit numerischen Daten funktioniert, werden mittels
# One-Hot-Endcoding aus den kategorischen Spalten, numerische Daten generiert
X_dumm = pd.get_dummies(half_apps[categorical_cols])

In [13]:
X_dumm.head(2)

,Installs_0+,Installs_1+,"Installs_1,000+","Installs_1,000,000+","Installs_1,000,000,000+",Installs_10+,"Installs_10,000+","Installs_10,000,000+",Installs_100+,"Installs_100,000+",...,Currency_USD,Currency_VND,Currency_XXX,Currency_ZAR,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
1079365,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
788722,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [14]:
# Zusammenfügen beider numerischen Listen
X = pd.concat([half_apps[numerical_cols], X_dumm], axis = 1)

In [15]:
X.head()

,Rating,Rating Count,Minimum Installs,Maximum Installs,Free,Price,Size,Ad Supported,In App Purchases,Editors Choice,...,Currency_USD,Currency_VND,Currency_XXX,Currency_ZAR,Content Rating_Adults only 18+,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
1079365,3.3,29.0,5000.0,8245.0,1.0,0.0,2.9,0.0,1.0,0.0,...,1,0,0,0,0,1,0,0,0,0
788722,4.0,7.0,1000.0,2369.0,1.0,0.0,34.0,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
2109951,0.0,0.0,1.0,2.0,1.0,0.0,2.9,1.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
807712,4.0,10.0,1000.0,1436.0,1.0,0.0,9.5,0.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0
1895366,0.0,0.0,1000.0,1836.0,1.0,0.0,17.0,1.0,0.0,0.0,...,1,0,0,0,0,1,0,0,0,0


In [16]:
y = half_apps['Category'] #.copy()
print(f"X und y haben gleiche Anzahl: {X.shape[0] == y.shape[0]}")

X und y haben gleiche Anzahl: True


In [17]:
label_encoder = LabelEncoder()

In [18]:
y = label_encoder.fit_transform(y) # macht alles zu 0, 1, 2,3 ...

In [19]:
# Daten in Trainings- und Test aufteilen
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Testdaten in Entwicklung und echten Test aufteilen (50-50, stratifiziert)
X_dev, X_test, y_dev, y_test = train_test_split(X_test1, y_test1, test_size=0.5, stratify=y_test1, random_state=42)

- Gridsearch durchführen
- manuelle Vorhersagen ebenso durchführen

In [21]:
# Parameter des Algorithmus setzen
# Pipeline neu definieren - ohne Parametersetzen 

feature_selection=SelectFromModel(LinearSVC(penalty="l1", dual=False))
classifier = XGBClassifier()

pipeline = Pipeline([('feature_selection', feature_selection), ('classifier', classifier)])

# Parameterraum definieren: key ist schrittname__parametername, value die zu prüfenden Werte

parameters = {  
    'feature_selection__threshold': (None, 'mean'), 
    'classifier__booster': ('gbtree','gblinear'),
}

# Suche über den gesamten Parameterraum (cross validation über die Trainingsdaten)
grid_search = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

grid_search.fit(X_train[:50000], y_train[:50000])

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START classifier__booster=gbtree, feature_selection__threshold=None


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:09:51] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 1/4] END classifier__booster=gbtree, feature_selection__threshold=None;, score=0.195 total time= 1.8min
[CV 2/5; 1/4] START classifier__booster=gbtree, feature_selection__threshold=None


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:11:35] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 1/4] END classifier__booster=gbtree, feature_selection__threshold=None;, score=0.202 total time= 1.7min
[CV 3/5; 1/4] START classifier__booster=gbtree, feature_selection__threshold=None


...

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:36:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 4/4] END classifier__booster=gblinear, feature_selection__threshold=mean;, score=0.139 total time= 1.1min


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:38:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(estimator=Pipeline(steps=[('feature_selection',
                                        SelectFromModel(estimator=LinearSVC(dual=False,
                                                                            penalty='l1'))),
                                       ('classifier',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      enable_categorical=False,
                                                      gamma=None, gpu_id=None,
                                                      importance_type=None,
                                                      interaction_constraints=None,
              

In [22]:
# Welche Parameterkombination ist die beste?
print(grid_search.best_estimator_)

Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
        

In [23]:
# Pipeline für die beste Feature-Kombination definieren
# Parameter aus dem .best_estimator Ergebnis entnehmen
final_pipeline = Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, 
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=None, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

train_labels = cross_val_predict(final_pipeline, X_train[:50000] , y_train[:50000], cv=10)

# Precision/Recall/F-Wert berechnen

print(classification_report(y_train[:50000], train_labels[:50000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:40:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:42:11] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[16:54:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:56:16] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:58:13] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.16      0.09      0.11       572
           1       0.02      0.00      0.00       472
           2       0.04      0.00      0.00       410
           3       0.04      0.01      0.01       275
           4       0.20      0.22      0.21      3006
           5       0.17      0.42      0.24      3593
           6       0.09      0.03      0.04       212
           7       0.18      0.19      0.19      1264
           8       0.13      0.02      0.04      1128
           9       0.19      0.35      0.25      5966
          10       0.17      0.17      0.17      3464
          11       0.04      0.00      0.01       334
          12       0.18      0.09      0.12 

### Vorhersagen auf den Trainingsdaten

In [24]:
# Parameter des Algorithmus setzen
pipeline1 = Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, 
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=None, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

pipeline1.fit(X_train[:50000], y_train[:50000])

train_labels1 = pipeline1.predict(X_train[:50000])
      
print(classification_report(y_train[:50000], train_labels1[:50000]))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:00:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.76      0.51      0.61       572
           1       0.90      0.16      0.27       472
           2       0.96      0.17      0.29       410
           3       0.90      0.41      0.56       275
           4       0.39      0.41      0.40      3006
           5       0.23      0.59      0.33      3593
           6       0.96      0.54      0.69       212
           7       0.45      0.47      0.46      1264
           8       0.77      0.17      0.27      1128
           9       0.30      0.52      0.38      5966
          10       0.36      0.37      0.36      3464
          11       0.79      0.14      0.24       334
          12       0.62      0.28      0.38 

### Vorhersage auf den Entwicklungsdaten --> X_dev

In [25]:
# Parameter des Algorithmus setzen
pipeline5 = Pipeline(steps=[('feature_selection',
                 SelectFromModel(estimator=LinearSVC(dual=False,
                                                     penalty='l1'))),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, 
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=None, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])
pipeline5.fit(X_train[:50000], y_train[:50000])

print("Default-Score des Klassifizierers: Accuracy=",pipeline5.score(X_test1, y_test1), "\n")

train_labels5 = pipeline5.predict(X_dev)
      
print(classification_report(y_dev, train_labels5))

/home/user/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:02:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Default-Score des Klassifizierers: Accuracy= 0.205914168678899 

              precision    recall  f1-score   support

           0       0.20      0.12      0.15      1132
           1       0.02      0.00      0.00       915
           2       0.04      0.00      0.00       851
           3       0.02      0.00      0.01       510
           4       0.21      0.23      0.22      5793
           5       0.17      0.43      0.24      6848
           6       0.08      0.03      0.04       398
           7       0.19      0.20      0.19      2445
           8       0.18      0.03      0.05      2331
           9       0.19      0.34      0.24     11779
          10       0.18      0.18      0.18      6761
          11       0.14      0.01

### Auswertung Trainings- vs. Testdaten
- grundsätzlich haben die Trainingsdaten eine höhere Accuracy als die Entwicklungs- bzw. Testdaten
----
Trotz zufälliger Halbierung des Datensatzes, Eliminierung von NaN-Werten und Eingrenzung der Zielklassen, können aufgrund der zahlreichen Datensätze keine Schlüsse gezogen werden. Die Accuracy liegt bei allen Vorhersagen recht niedrig, da bei vielen Zielklassen die Precision und F-score `0.0` bewertet wurden und zu den jeweiligen Zielklassen keine aussagekräftigen Trainings- oder Testdaten zugeordnet werden konnten.